# Movie recommendation on the basis of user past data on movie ratings
if movie rating is high for a particular kind of movie then similar movies would be recommended
if movie rating is low then movies which are negetaively correlated are recommended

In [21]:
#Import Libraries

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import re

In [22]:
# For this approach i would need a dataset that contains users and ratings data

In [23]:
movies = pd.read_csv('temp_data/movie.csv', usecols=['movieId','title','genres'])

In [24]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [25]:
ratings = pd.read_csv('temp_data/rating.csv', usecols=['userId','movieId','rating'])

In [26]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [27]:
ratings = pd.merge(ratings,movies, on='movieId', how='left')

In [28]:
ratings.head(5)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [29]:
#making a pivot table with index user , columns as movies title and values as ratings so that i can find 
# correlation between movies

In [30]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')

In [31]:
#drop movies which are rated by less than 10 users
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)

In [10]:
userRatings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.5,3.5,0.0


In [ ]:
#using cosine similarity to find cosine distance between movies

In [ ]:
#taking transpose of userRatings so that movies becomes the rows

In [11]:
similarity = cosine_similarity(userRatings.T)

In [12]:
sim_matrix = pd.DataFrame(similarity, index=userRatings.columns, columns=userRatings.columns)

In [13]:
sim_matrix

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.099735,0.000000,0.180401,0.036417,0.125357,0.254537,0.078676,0.033942,0.034303,...,0.045145,0.055071,0.155553,0.184692,0.140750,0.091918,0.038152,0.211467,0.089634,0.372876
(500) Days of Summer (2009),0.099735,1.000000,0.173843,0.326122,0.225255,0.206167,0.198959,0.222581,0.169654,0.233237,...,0.401963,0.208675,0.104340,0.450463,0.402251,0.306076,0.260308,0.097935,0.276512,0.169385
10 Cloverfield Lane (2016),0.000000,0.173843,1.000000,0.036935,0.133272,0.045497,0.022956,0.074102,0.000000,0.291018,...,0.262842,0.119092,0.000000,0.297371,0.271721,0.227363,0.341125,0.200462,0.121770,0.032083
10 Things I Hate About You (1999),0.180401,0.326122,0.036935,1.000000,0.276708,0.282549,0.270490,0.095586,0.132512,0.088857,...,0.280660,0.142059,0.169472,0.154749,0.227712,0.340590,0.110769,0.167085,0.176720,0.160426
"10,000 BC (2008)",0.036417,0.225255,0.133272,0.276708,1.000000,0.265326,0.155047,0.102822,0.000000,0.113494,...,0.281309,0.109188,0.117015,0.215190,0.274656,0.272574,0.126702,0.114869,0.226588,0.112168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoolander (2001),0.091918,0.306076,0.227363,0.340590,0.272574,0.246544,0.328696,0.196745,0.062037,0.134348,...,0.338465,0.273801,0.038226,0.294500,0.357185,1.000000,0.165195,0.144717,0.371858,0.159466
Zootopia (2016),0.038152,0.260308,0.341125,0.110769,0.126702,0.110579,0.131859,0.119460,0.096690,0.254504,...,0.314808,0.184455,0.045615,0.257177,0.343138,0.165195,1.000000,0.086647,0.234282,0.064098
eXistenZ (1999),0.211467,0.097935,0.200462,0.167085,0.114869,0.094899,0.130010,0.053689,0.032119,0.155810,...,0.118802,0.055587,0.191358,0.184123,0.118236,0.144717,0.086647,1.000000,0.192259,0.170341


In [ ]:
#multiplying movie rating with rating(given)-2.5 so that if rating is low then negatively retlated movies pops up
# and we can recommend opposite kind of movies
#and if rating is high there would be no effect, positively correlated would remain on the top and negatively 
# correlation become more negative

In [14]:
class recom_movie():
    def __init__(self,sim_matrix):
        self.sim_matrix = sim_matrix
    def recommend_movie(self,movie, rating):
        recom = (self.sim_matrix[movie]*(rating-2.5)).sort_values(ascending=False)
        return(list(recom[0:5].index.values))
        

In [15]:
rm = recom_movie(sim_matrix)


In [ ]:
#results of recommendation

In [16]:
rm.recommend_movie("Amazing Spider-Man, The (2012)",5)

['Amazing Spider-Man, The (2012)',
 'Captain America: The First Avenger (2011)',
 'Thor (2011)',
 'Star Trek Into Darkness (2013)',
 'Wreck-It Ralph (2012)']

In [17]:
rm.recommend_movie("Amazing Spider-Man, The (2012)",1)

['Father of the Bride (1950)',
 'Drop Zone (1994)',
 'Room with a View, A (1986)',
 'Conquest of the Planet of the Apes (1972)',
 'Maltese Falcon, The (a.k.a. Dangerous Female) (1931)']

In [18]:
rm.recommend_movie("(500) Days of Summer (2009)",5)

['(500) Days of Summer (2009)',
 'Silver Linings Playbook (2012)',
 'Up in the Air (2009)',
 'Adventureland (2009)',
 '50/50 (2011)']

In [19]:
rm.recommend_movie("Toy Story 3 (2010)",1)

['Jean de Florette (1986)',
 'Shadowlands (1993)',
 'Persepolis (2007)',
 'Goodbye Girl, The (1977)',
 'Take the Money and Run (1969)']

In [20]:
rm.recommend_movie("Aliens (1986)",1)

['The Theory of Everything (2014)',
 'Marley & Me (2008)',
 'Twilight Saga: Eclipse, The (2010)',
 'Warrior (2011)',
 'Wedding Date, The (2005)']

In [ ]:
# Steps followed:
# in this notebook i have tried to recommend the movies based on past user data i.e past ratings
# if high rating is given to some kind of movie then similar movies would be recommended
# if low rating is given to some kind of movie then opposite kind of movies would be recommended
# method to find similar movies:
#     based on users ratings it is decided what kind of movie , for ex if there is some thriller than users who like 
#     thriller movies give high rating to it , other users will give low rating to it.
#     so that movie has data of both the users who like it and who dont like it as a single column in dataset.
   
    
#     now cosine distance is computed in order to find movies which are similar by considering movies as vectors 
#     and user ratings as dimensions
    
# method to recommend movies:
#     we will pass movie id and rating to our function and it will multiply the series of movie ratings with 
#     rating-2.5 so that if rating is low then negatively retlated movies pops up
#     and we can recommend opposite kind of movies
#     and if rating is high there would be no effect, positively correlated would remain on the top and negatively 
#     correlation become more negative

In [ ]:
# Limitations of this approach:
#     This approach heavily depends on the ratings given by users, if it is not accurate then our function 
#     does't give accurate results